In [3]:
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import mdtraj as md
# import nglview as ngl

In [4]:
c_in = md.load('./compare.gro')
s_in = md.load('./similar.gro')

In [88]:
c_prot = c_in.topology.select("resid 0 to 19")
s_prot = s_in.topology.select("resid 0 to 19")
c_wat = c_in.topology.select("waters and name O")
s_wat = s_in.topology.select("waters and name O")

In [117]:
def prune(trj, d=0.5):
    prot = trj.topology.select("resid 0 to 19")
    wat = trj.topology.select("waters and name O")
    pairs = np.stack([np.array([prot]*len(wat)).T.flatten(),np.array([wat]*len(prot)).flatten()]).T
    dist = md.compute_distances(trj,pairs)[0]
    idx = np.where(dist <= d)[0]
    wat_atoms = pairs[idx,1]
    wat_res = [trj.topology.atom(i).residue.index for i in wat_atoms]
    atom_sel = np.unique(list(prot) + [j.index  for i in wat_res for j in trj.topology.residue(i).atoms  ])
    return trj.atom_slice(atom_sel)

In [121]:
prune(c_in, 0.4).save('compare_prune.pdb')
prune(s_in, 0.4).save('similar_prune.pdb')

In [89]:
pairs = np.stack([np.array([c_prot]*len(c_wat)).T.flatten(),np.array([c_wat]*len(c_prot)).flatten()]).T
dist = md.compute_distances(c_in,pairs)[0]
idx = np.where(dist <= 0.5)[0]
wat_atoms = pairs[idx,1]

In [90]:
c_wat_res = [c_in.topology.atom(i).residue.index for i in wat_atoms]

In [114]:
c_atom_sel = np.unique(list(c_prot) + [j.index  for i in c_wat_res for j in c_in.topology.residue(i).atoms  ])

In [116]:
c_in.atom_slice(c_atom_sel).save('compare_prune.pdb')

In [19]:
np.stack([np.array([c_prot]*len(c_wat)).T.flatten(),np.array([c_wat]*len(c_prot)).flatten()])

array([[   14,    14,    14, ...,   282,   282,   282],
       [  332,   333,   334, ..., 35305, 35306, 35307]])

In [17]:
np.array([c_prot]*len(c_wat)).shape

(34976, 269)

In [16]:
np.array([c_wat]*len(c_prot)).shape

(269, 34976)